# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating above 4.5?
- How many businesses have a rating below 3?
- Return the text of the oldest review in the table.
- Return the overall rating of the business with the oldest review. 
- Find the highest rated business and return text of the newest review of the three.
- Find the lowest rated business and return text of the newest review of the three.  


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [38]:
## Import everything we need

import mysql.connector
from mysql.connector import errorcode
import config
from datetime import date, datetime, timedelta
import requests
import time

In [39]:
## Connect to DB server on AWS

def connect_to_AWS():
    cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        passwd = config.password
    )
    return cnx

## Close the connection
def close_connections():
    cursor.close()
    conn.close()
    
## Create the connection and cursor
conn = connect_to_AWS()
cursor = conn.cursor()
db_name = "Yelp_API"

In [80]:
## Function to close the connections
# close_connections()

In [40]:
## Function to create new DB 
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)

## Try to use the database we are creating.
## If we can't use it, then that means the database doesn't exist
    ## Thus we create the database and switch over to it.
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        conn.database = db_name
    else:
        print(err)
        exit(1)

In [41]:
# Function to create tables from a dictionary of tables
# dictionary will have the name of the table as the key and the query to create that table as the value
def create_table(dict_of_tables):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")


In [42]:
TABLES = {}
TABLES

{}

In [43]:
# Create a table for the Businesses
TABLES['Businesses'] = (
    "CREATE TABLE Businesses ("
    "  businessId varchar(30) NOT NULL,"
    "  businessName varchar(255),"
    "  price varchar(4),"
    "  rating REAL,"
    "  PRIMARY KEY (businessId)"
    ") ENGINE=InnoDB")

In [44]:
# Create a table for the reviews
TABLES['Reviews'] = ("""
    CREATE TABLE Reviews (
        reviewId varchar(30) NOT NULL,
        businessId varchar(30),
        rating INT,
        text varchar(255),
        time_created DATETIME,
        PRIMARY KEY (reviewId),
        FOREIGN KEY (businessId)
            REFERENCES Businesses(businessId)
            ON DELETE CASCADE
    ) ENGINE=InnoDB
""")

In [45]:
## Drop all tables to refresh
def drop_everything():
    cursor.execute("""DROP TABLE Reviews;""")
    conn.commit()
    cursor.execute("""DROP TABLE Businesses;""")
    conn.commit()

In [46]:
# drop_everything()

In [47]:
create_table(TABLES)

Creating table Businesses: already exists.
Creating table Reviews: already exists.


## Part 2: Create ETL pipeline for the business data from the API

In [48]:
#this is where all of our global variables are
key = config.apiKey
term = 'bars'
location = 'LA'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50,
                'offset' : 0
             }
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(key)}


In [16]:
## Function to convert our string to a datetime object
def string_to_time(string):
    return datetime.strptime(string,"%Y-%m-%d %H:%M:%S")

# write a function to parse the API response 
# so that you can easily insert the data in to the DB

## A function that parses the API response
## Uploads businesses into our database
    ## For each business, also get its 3 reviews and upload them
def all_business_results(url_params):
    response = requests.get(url, headers=headers, params=url_params)
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.extend(yelp_call_businesses(url_params))
        add_to_businesses(results[cur:])
        time.sleep(1) #Wait a second
        cur += 50
    return results

# write a function to make a call to the DB 
# updates response with new parameters
def yelp_call_businesses(url_params):
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data

## Function to upload businesses and reviews to the database
## Takes in a list of dictionaries representing our businesses
def add_to_businesses(list_of_businesses):
    for business in list_of_businesses:
        cursor.execute("""
            INSERT INTO Businesses (businessId, businessName, price, rating)
            VALUES (%s, %s, %s, %s)
        """,(business['id'],business['name'],return_key(business,'price'),business['rating']))
        conn.commit()
        ## Function to get the three reviews for the current business
        get_all_reviews(business['id'])
        

# write a function to parse the API response 
# so that you can easily insert the data in to the DB

## Function to get all three reviews for a given businessId
def get_all_reviews(business_id):
    url_2 = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
    review_response = requests.get(url_2, headers=headers)
    reviews = review_response.json()['reviews']
    for review in reviews:
        ## Add the reviews to the review table
        add_to_reviews(review,business_id)
        
        
## Function to add the reviews to the table
def add_to_reviews(review,business_id):
    cursor.execute("""
           `businessId, rating, text, time_created)
            VALUES (%s, %s, %s, %s, %s)""", 
            (review['id'], business_id, review['rating'], return_key(review,'text'), string_to_time(review['time_created'])))
    conn.commit()

## Check if a key exists in a dictionary.
    ## Returns empty string if the key doesn't exist.
    ## Returns the value of the key if it does exist.
def return_key(dictionary, check_key):
    if check_key in dictionary.keys():
        return dictionary[check_key]
    else:
        return ''
    


In [37]:
# first_50 = yelp_call_businesses(url_params)

In [68]:
# all_businesses

[{'id': 'XG49PgfCvT3XPfHIJ4NIhQ',
  'alias': 'paper-tiger-bar-los-angeles',
  'name': 'Paper Tiger Bar',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/lt6bk_cW8k2A5cubSU5nUw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/paper-tiger-bar-los-angeles?adjust_creative=0U31FNv6hJ-Z7ZRgG8QRwg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=0U31FNv6hJ-Z7ZRgG8QRwg',
  'review_count': 140,
  'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
   {'alias': 'lounges', 'title': 'Lounges'}],
  'rating': 4.0,
  'coordinates': {'latitude': 34.0761251, 'longitude': -118.30877},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '4574 Beverly Blvd',
   'address2': None,
   'address3': '',
   'city': 'Los Angeles',
   'zip_code': '90004',
   'country': 'US',
   'state': 'CA',
   'display_address': ['4574 Beverly Blvd', 'Los Angeles, CA 90004']},
  'phone': '+13238443716',
  'display_phone': '(323) 844-3716',
  'distance': 1

In [17]:
## Run the code!
## and store the list of businesses just in case
all_businesses = all_business_results(url_params)

3400 total matches found.


In [ ]:
# Write a function to take your parsed data and insert it into the DB

## Part 3: Create ETL pipeline for the review data from the API

In [ ]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that take a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# # Your code here; use a function or loop to retrieve all the results from your original request
# import time

# def yelp_call(url_params, api_key):
    
#     response = requests.get(url, headers=headers, params=url_params)
    
#     data = response.json()['businesses']
#     return data

# def all_results(url_params, api_key):
#     num = response.json()['total']
#     print('{} total matches found.'.format(num))
#     cur = 0
#     results = []
#     while cur < num and cur < 1000:
#         url_params['offset'] = cur
#         results.append(yelp_call(url_params, api_key))
#         time.sleep(1) #Wait a second
#         cur += 50
#     return results

# term = 'pizza'
# location = 'Astoria NY'
# url_params = {  'term': term.replace(' ', '+'),
#                 'location': location.replace(' ', '+'),
#                 'limit' : 50
#              }
# df = all_results(url_params, api_key)
# print(len(df))
# df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```